# Data Preprocessing

Data sources:

* Ufo Sightings: Sigmond Axel. (2014). ufo-reports (Version commit-c0915f18186e5e2227083702049a838258001a2a) [Data set]. Zenodo. http://doi.org/10.5281/zenodo.1205624
* Alcohol Consumption: OECD (2018), Alcohol consumption (indicator). doi: 10.1787/e6895909-en (Accessed on 22 March 2018) via https://data.oecd.org/healthrisk/alcohol-consumption.htm

In [1]:
RAW_DATA_DIR = '../data/raw'
PROCESSED_DATA_FILENAME = '../data/processed/ufo_alcohol.csv'

UFO_SIGHTINGS = RAW_DATA_DIR + '/ufo-scrubbed-geocoded-time-standardized.csv'
ALC_CONSUMPTION = RAW_DATA_DIR + '/DP_LIVE_22032018202902423.csv'

In [2]:
import pandas as pd
from IPython.display import display

## UFO SIGHTINGS

We want to aggregate the number of sightings per year in the USA. Since the original data contains a single sighting per row, we will have to extract the year and aggregate the data ourselves.

In [3]:
dateparse = lambda date: pd.datetime.strptime(date.replace('24:', '00:'), '%m/%d/%Y %H:%M')

ufo_df = pd.read_csv(UFO_SIGHTINGS, header=None, low_memory=False, 
                 parse_dates=['datetime'], date_parser=dateparse,
                 names=['datetime', 'city', 'state', 'country','shape',
                        'duration (seconds)','duration (hours/min)',
                        'comments','date posted','latitude','longitude'])

We only want to count sightings in the USA:

In [4]:
ufo_df = ufo_df.loc[ufo_df['country'] == 'us']

Before doing that, we'll take a look at the data:

In [5]:
display(ufo_df.head())

,datetime,city,state,country,shape,duration (seconds),duration (hours/min),comments,date posted,latitude,longitude
0,1949-10-10 20:30:00,san marcos,tx,us,cylinder,2700,45 minutes,This event took place in early fall around 194...,4/27/2004,29.8830556,-97.941111
3,1956-10-10 21:00:00,edna,tx,us,circle,20,1/2 hour,My older brother and twin sister were leaving ...,1/17/2004,28.9783333,-96.645833
4,1960-10-10 20:00:00,kaneohe,hi,us,light,900,15 minutes,AS a Marine 1st Lt. flying an FJ4B fighter/att...,1/22/2004,21.4180556,-157.803611
5,1961-10-10 19:00:00,bristol,tn,us,sphere,300,5 minutes,My father is now 89 my brother 52 the girl wit...,4/27/2007,36.5950000,-82.188889
7,1965-10-10 23:45:00,norwalk,ct,us,disk,1200,20 minutes,A bright orange color changing to reddish colo...,10/2/1999,41.1175000,-73.408333


Extract **year** and add it as a separate column

In [6]:
ufo_df['year'] = ufo_df['datetime'].dt.year

Only keep year column

In [7]:
ufo_df = ufo_df [['year']]

Count number of occurences per year and state

In [8]:
original = ufo_df.copy()
ufo_df = pd.DataFrame({'ufo_sightings' : original.groupby( [ 'year'] )['year'].count()}).reset_index()

Our dataframe now has the data we need

In [9]:
display(ufo_df.tail())

,year,ufo_sightings
78,2010,3548
79,2011,4379
80,2012,6320
81,2013,6056
82,2014,1964


In [10]:
ufo_df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
year,83.0,1972.240964,25.503988,1910.0,1952.5,1973.0,1993.5,2014.0
ufo_sightings,83.0,784.506024,1431.080799,1.0,29.5,147.0,291.0,6320.0


## ALCOHOL CONSUMPTION

The alcohol consumption dataset wont need any pre-processing. We're just going to remove/rename some columns

In [11]:
alc_df = pd.read_csv(ALC_CONSUMPTION, low_memory=False)

Take a look at the original dataset

In [12]:
display(alc_df.head())

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,ALCOHOL,TOT,LT_CAP15,A,1960,9.3,NaN
1,AUS,ALCOHOL,TOT,LT_CAP15,A,1961,9.4,NaN
2,AUS,ALCOHOL,TOT,LT_CAP15,A,1962,9.5,NaN
3,AUS,ALCOHOL,TOT,LT_CAP15,A,1963,9.8,NaN
4,AUS,ALCOHOL,TOT,LT_CAP15,A,1964,10.1,NaN


We only need data from the USA

In [13]:
alc_df = alc_df.loc[alc_df['LOCATION'] == 'USA']

Pick columns of interest

In [14]:
alc_df = alc_df [['TIME', 'Value']]

Rename columns

In [15]:
alc_df = alc_df.rename(columns={'TIME': 'year', 'Value': 'alcohol_consumption'})

In [16]:
display(alc_df.head())

,year,alcohol_consumption
1555,1960,7.8
1556,1961,7.8
1557,1962,8.0
1558,1963,8.1
1559,1964,8.4


In [17]:
alc_df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
year,55.0,1987.000000,16.020820,1960.0,1973.5,1987.0,2000.5,2014.0
alcohol_consumption,55.0,9.043636,0.799945,7.8,8.4,8.8,9.8,10.4


## Merge both datasets

Merge both datasets into a single one by only selecting years with data available on both datasets.

In [18]:
merged_df = pd.merge(ufo_df, alc_df, on='year')

In [19]:
display(merged_df.head())

,year,ufo_sightings,alcohol_consumption
0,1960,51,7.8
1,1961,41,7.8
2,1962,56,8.0
3,1963,68,8.1
4,1964,78,8.4


Save merged dataset

In [20]:
merged_df.to_csv(PROCESSED_DATA_FILENAME, sep=',', index=False)